# Dreambooth
### Notebook implementation by Joe Penna (@MysteryGuitarM on Twitter) - Improvements by David Bielejeski

### Instructions
- Sign up for RunPod here: https://runpod.io/?ref=n8yfwyum
    - Note: That's my personal referral link. Please don't use it if we are mortal enemies.

- Click *Deploy* on either `SECURE CLOUD` or `COMMUNITY CLOUD`

- Follow the rest of the instructions in this video: https://www.youtube.com/watch?v=7m__xadX0z0#t=5m33.1s

Latest information on:
https://github.com/JoePenna/Dreambooth-Stable-Diffusion

In [2]:
# BUILD ENV
!pip install omegaconf
!pip install einops
!pip install pytorch-lightning==1.6.5
!pip install test-tube
!pip install transformers
!pip install kornia
!pip install -e git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
!pip install -e git+https://github.com/openai/CLIP.git@main#egg=clip
!pip install setuptools==59.5.0
!pip install pillow==9.0.1
!pip install torchmetrics==0.6.0
!pip install -e .
!pip install protobuf==3.20.1
!pip install gdown
!pip install -qq diffusers["training"]==0.3.0 transformers ftfy
!pip install -qq "ipywidgets>=7,<8"
!pip install huggingface_hub
!pip install ipywidgets==7.7.1
!pip install captionizer==1.0.1

!pip install torch==1.11.0+cu113 torchvision==0.12.0+cu113 torchaudio==0.11.0 --extra-index-url https://download.pytorch.org/whl/cu113


Obtaining taming-transformers from git+https://github.com/CompVis/taming-transformers.git@master#egg=taming-transformers
  Updating c:\users\mail\dreambooth-stable-diffusion\src\taming-transformers clone (to revision master)
  Attempting uninstall: taming-transformers
    Found existing installation: taming-transformers 0.0.1
    Uninstalling taming-transformers-0.0.1:
      Successfully uninstalled taming-transformers-0.0.1
  Running setup.py develop for taming-transformers


  Running command git fetch -q --tags
  Running command git reset --hard -q 3ba01b241669f5ade541ce990f7650a3b8f65318


Obtaining clip from git+https://github.com/openai/CLIP.git@main#egg=clip
  Updating c:\users\mail\dreambooth-stable-diffusion\src\clip clone (to revision main)


  Running command git fetch -q --tags
  Running command git reset --hard -q d50d76daa670286dd6cacf3bcd80b5e4823fc8e1


  Attempting uninstall: clip
    Found existing installation: clip 1.0
    Uninstalling clip-1.0:
      Successfully uninstalled clip-1.0
  Running setup.py develop for clip
Obtaining file:///C:/Users/mail/Dreambooth-Stable-Diffusion
  Attempting uninstall: latent-diffusion
    Found existing installation: latent-diffusion 0.0.1
    Uninstalling latent-diffusion-0.0.1:
      Successfully uninstalled latent-diffusion-0.0.1
  Running setup.py develop for latent-diffusion


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113
  Attempting uninstall: torch
    Found existing installation: torch 1.13.0
    Uninstalling torch-1.13.0:
      Successfully uninstalled torch-1.13.0
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.14.0
    Uninstalling torchvision-0.14.0:
      Successfully uninstalled torchvision-0.14.0


In [3]:
# Hugging Face Login
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved in your configured git credential helpers (manager,manager-core).
Your token has been saved to C:\Users\mail\.huggingface\token
Login successful


In [4]:
# Download the 1.4 sd model
from IPython.display import clear_output

from huggingface_hub import hf_hub_download
downloaded_model_path = hf_hub_download(
 repo_id="CompVis/stable-diffusion-v-1-4-original",
 filename="sd-v1-4.ckpt",
 use_auth_token=True
)

# Move the sd-v1-4.ckpt to the root of this directory as "model.ckpt"
actual_locations_of_model_blob = !readlink -f {downloaded_model_path}
!mv {actual_locations_of_model_blob[-1]} model.ckpt
clear_output()
print("✅ model.ckpt successfully downloaded")

✅ model.ckpt successfully downloaded


Training teaches your new model both your token **but** re-trains your class simultaneously.

From cursory testing, it does not seem like reg images affect the model too much. However, they do affect your class greatly, which will in turn affect your generations.

You can either generate your images here, or use the repos below to quickly download 1500 images.

In [3]:
# GENERATE 200 images - Optional
self_generated_files_prompt = "person" #@param {type:"string"}
self_generated_files_count = 200 #@param {type:"integer"}

!python scripts/stable_txt2img.py \
 --seed 10 \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter {self_generated_files_count} \
 --scale 10.0 \
 --ddim_steps 50 \
 --ckpt model.ckpt \
 --prompt {self_generated_files_prompt}

dataset=self_generated_files_prompt





Loading model from model.ckpt
Global Step: 470000
LatentDiffusion: Running in eps-prediction mode
DiffusionWrapper has 859.52 M params.
making attention of type 'vanilla' with 512 in_channels
Working with z of shape (1, 4, 32, 32) = 4096 dimensions.
making attention of type 'vanilla' with 512 in_channels
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps

Global seed set to 10
Some weights of the model checkpoint at openai/clip-vit-large-patch14 were not used when initializing CLIPTextModel: ['vision_model.encoder.layers.18.layer_norm1.weight', 'vision_model.encoder.layers.9.mlp.fc1.weight', 'vision_model.encoder.layers.5.self_attn.q_proj.bias', 'vision_model.encoder.layers.1.mlp.fc1.bias', 'vision_model.encoder.layers.0.self_attn.v_proj.bias', 'vision_model.encoder.layers.6.layer_norm1.weight', 'vision_model.encoder.layers.12.mlp.fc2.bias', 'vision_model.encoder.layers.16.self_attn.q_proj.weight', 'vision_model.encoder.layers.2.self_attn.out_proj.bias', 'vision_model.encoder.layers.13.self_attn.out_proj.bias', 'vision_model.encoder.layers.5.layer_norm2.weight', 'vision_model.encoder.layers.6.self_attn.q_proj.weight', 'vision_model.encoder.layers.3.mlp.fc1.weight', 'vision_model.encoder.layers.10.self_attn.q_proj.bias', 'vision_model.encoder.layers.13.layer_norm2.bias', 'vision_model.encoder.layers.17.layer_norm1.weight', 'vision_model.


Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 64, 64), eta 0.0
Running DDIM Sampling with 50 timesteps
Data shape for DDIM sampling is (1, 4, 




DDIM Sampler:  88%|########8 | 44/50 [00:14<00:01,  4.22it/s]


DDIM Sampler:  90%|######### | 45/50 [00:14<00:01,  4.04it/s]


DDIM Sampler:  92%|#########2| 46/50 [00:14<00:00,  4.07it/s]


DDIM Sampler:  94%|#########3| 47/50 [00:14<00:00,  4.14it/s]


DDIM Sampler:  96%|#########6| 48/50 [00:15<00:00,  4.18it/s]


DDIM Sampler:  98%|#########8| 49/50 [00:15<00:00,  4.17it/s]


DDIM Sampler: 100%|##########| 50/50 [00:15<00:00,  3.21it/s]


data: 100%|##########| 1/1 [00:19<00:00, 19.28s/it]

Sampling:   0%|          | 1/200 [00:19<1:03:56, 19.28s/it]

data:   0%|          | 0/1 [00:00<?, ?it/s]


DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]


DDIM Sampler:   2%|2         | 1/50 [00:00<00:13,  3.75it/s]


DDIM Sampler:   4%|4         | 2/50 [00:00<00:12,  3.85it/s]


DDIM Sampler:   6%|6         | 3/50 [00:00<00:12,  3.67it/s]


DDIM Sampler:   8%|8         | 4/50 [00:01<00:12,  3.71it/s]


DDIM Sampler:  10%|#         | 5/50 [00:01<00:11,  3.81it/s]


DDIM Sampler:  12%|

DDIM Sampler:  20%|##        | 10/50 [00:02<00:09,  4.09it/s]


DDIM Sampler:  22%|##2       | 11/50 [00:02<00:09,  4.15it/s]


DDIM Sampler:  24%|##4       | 12/50 [00:03<00:09,  4.19it/s]


DDIM Sampler:  26%|##6       | 13/50 [00:03<00:08,  4.20it/s]


DDIM Sampler:  28%|##8       | 14/50 [00:03<00:08,  4.16it/s]


DDIM Sampler:  30%|###       | 15/50 [00:03<00:08,  4.18it/s]


DDIM Sampler:  32%|###2      | 16/50 [00:03<00:08,  4.21it/s]


DDIM Sampler:  34%|###4      | 17/50 [00:04<00:07,  4.23it/s]


DDIM Sampler:  36%|###6      | 18/50 [00:04<00:07,  4.23it/s]


DDIM Sampler:  38%|###8      | 19/50 [00:04<00:07,  4.24it/s]


DDIM Sampler:  40%|####      | 20/50 [00:04<00:07,  4.28it/s]


DDIM Sampler:  42%|####2     | 21/50 [00:05<00:06,  4.15it/s]


DDIM Sampler:  44%|####4     | 22/50 [00:05<00:06,  4.11it/s]


DDIM Sampler:  46%|####6     | 23/50 [00:05<00:06,  4.12it/s]


DDIM Sampler:  48%|####8     | 24/50 [00:05<00:06,  4.08it/s]


DDIM Sampler:  50%|#####     | 25/50 [00

...

DDIM Sampler:  50%|#####     | 25/50 [00:07<00:06,  3.58it/s]


DDIM Sampler:  52%|#####2    | 26/50 [00:07<00:06,  3.62it/s]


DDIM Sampler:  54%|#####4    | 27/50 [00:08<00:06,  3.57it/s]


DDIM Sampler:  56%|#####6    | 28/50 [00:08<00:06,  3.58it/s]


DDIM Sampler:  58%|#####8    | 29/50 [00:08<00:05,  3.56it/s]


DDIM Sampler:  60%|######    | 30/50 [00:08<00:05,  3.49it/s]


DDIM Sampler:  62%|######2   | 31/50 [00:09<00:05,  3.53it/s]


DDIM Sampler:  64%|######4   | 32/50 [00:09<00:05,  3.33it/s]


DDIM Sampler:  66%|######6   | 33/50 [00:09<00:05,  3.33it/s]


DDIM Sampler:  68%|######8   | 34/50 [00:10<00:04,  3.30it/s]


DDIM Sampler:  70%|#######   | 35/50 [00:10<00:05,  2.95it/s]


DDIM Sampler:  72%|#######2  | 36/50 [00:10<00:05,  2.78it/s]


DDIM Sampler:  74%|#######4  | 37/50 [00:11<00:04,  2.82it/s]


DDIM Sampler:  76%|#######6  | 38/50 [00:11<00:04,  2.81it/s]


DDIM Sampler:  78%|#######8  | 39/50 [00:11<00:03,  2.95it/s]


DDIM Sampler:  80%|########  | 40/50 [00



DDIM Sampler:  60%|######    | 30/50 [00:09<00:06,  3.15it/s]


DDIM Sampler:  62%|######2   | 31/50 [00:09<00:05,  3.26it/s]


DDIM Sampler:  64%|######4   | 32/50 [00:10<00:05,  3.05it/s]


DDIM Sampler:  66%|######6   | 33/50 [00:10<00:05,  3.04it/s]


DDIM Sampler:  68%|######8   | 34/50 [00:10<00:05,  3.04it/s]


DDIM Sampler:  70%|#######   | 35/50 [00:11<00:04,  3.15it/s]


DDIM Sampler:  72%|#######2  | 36/50 [00:11<00:04,  3.21it/s]


DDIM Sampler:  74%|#######4  | 37/50 [00:11<00:03,  3.28it/s]


DDIM Sampler:  76%|#######6  | 38/50 [00:12<00:03,  3.31it/s]


DDIM Sampler:  78%|#######8  | 39/50 [00:12<00:03,  3.08it/s]


DDIM Sampler:  80%|########  | 40/50 [00:12<00:03,  3.16it/s]


DDIM Sampler:  82%|########2 | 41/50 [00:13<00:02,  3.11it/s]


DDIM Sampler:  84%|########4 | 42/50 [00:13<00:02,  3.13it/s]


DDIM Sampler:  86%|########6 | 43/50 [00:13<00:02,  2.93it/s]


DDIM Sampler:  88%|########8 | 44/50 [00:14<00:02,  2.96it/s]


DDIM Sampler:  90%|######### | 45/50 [



DDIM Sampler:  84%|########4 | 42/50 [00:14<00:02,  2.92it/s]


DDIM Sampler:  86%|########6 | 43/50 [00:15<00:02,  2.83it/s]


DDIM Sampler:  88%|########8 | 44/50 [00:15<00:02,  2.77it/s]


DDIM Sampler:  90%|######### | 45/50 [00:15<00:01,  2.83it/s]


DDIM Sampler:  92%|#########2| 46/50 [00:16<00:01,  2.86it/s]


DDIM Sampler:  94%|#########3| 47/50 [00:16<00:01,  2.96it/s]


DDIM Sampler:  96%|#########6| 48/50 [00:16<00:00,  3.00it/s]


DDIM Sampler:  98%|#########8| 49/50 [00:17<00:00,  2.94it/s]


DDIM Sampler: 100%|##########| 50/50 [00:17<00:00,  2.85it/s]


data: 100%|##########| 1/1 [00:17<00:00, 17.85s/it]

Sampling:  32%|###2      | 65/200 [16:38<40:28, 17.99s/it]

data:   0%|          | 0/1 [00:00<?, ?it/s]


DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]


DDIM Sampler:   2%|2         | 1/50 [00:00<00:25,  1.90it/s]


DDIM Sampler:   4%|4         | 2/50 [00:00<00:22,  2.17it/s]


DDIM Sampler:   6%|6         | 3/50 [00:01<00:19,  2.47it/s]


DDIM Sampler:   8%|


DDIM Sampler:  96%|#########6| 48/50 [00:16<00:00,  3.04it/s]


DDIM Sampler:  98%|#########8| 49/50 [00:16<00:00,  3.06it/s]


DDIM Sampler: 100%|##########| 50/50 [00:17<00:00,  2.92it/s]


data: 100%|##########| 1/1 [00:17<00:00, 17.41s/it]

Sampling:  36%|###5      | 71/200 [18:22<37:18, 17.35s/it]

data:   0%|          | 0/1 [00:00<?, ?it/s]


DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]


DDIM Sampler:   2%|2         | 1/50 [00:00<00:19,  2.53it/s]


DDIM Sampler:   4%|4         | 2/50 [00:00<00:17,  2.77it/s]


DDIM Sampler:   6%|6         | 3/50 [00:01<00:17,  2.65it/s]


DDIM Sampler:   8%|8         | 4/50 [00:01<00:15,  3.00it/s]


DDIM Sampler:  10%|#         | 5/50 [00:01<00:15,  2.95it/s]


DDIM Sampler:  12%|#2        | 6/50 [00:02<00:15,  2.78it/s]


DDIM Sampler:  14%|#4        | 7/50 [00:02<00:14,  2.91it/s]


DDIM Sampler:  16%|#6        | 8/50 [00:02<00:13,  3.08it/s]


DDIM Sampler:  18%|#8        | 9/50 [00:03<00:13,  2.93it/s]


DDIM Sampler:  20%|##     

DDIM Sampler: 100%|##########| 50/50 [00:15<00:00,  3.32it/s]


data: 100%|##########| 1/1 [00:15<00:00, 15.41s/it]

Sampling:  38%|###8      | 77/200 [20:05<33:22, 16.28s/it]

data:   0%|          | 0/1 [00:00<?, ?it/s]


DDIM Sampler:   0%|          | 0/50 [00:00<?, ?it/s]


DDIM Sampler:   2%|2         | 1/50 [00:00<00:15,  3.12it/s]


DDIM Sampler:   4%|4         | 2/50 [00:00<00:14,  3.37it/s]

mv: target 'regularization_images/person' is not a directory

DDIM Sampler:   6%|6         | 3/50 [00:01<00:16,  2.90it/s]


DDIM Sampler:   8%|8         | 4/50 [00:01<00:16,  2.86it/s]


DDIM Sampler:  10%|#         | 5/50 [00:01<00:15,  2.91it/s]


DDIM Sampler:  12%|#2        | 6/50 [00:02<00:14,  2.97it/s]


DDIM Sampler:  14%|#4        | 7/50 [00:02<00:13,  3.18it/s]


DDIM Sampler:  16%|#6        | 8/50 [00:02<00:13,  3.12it/s]


DDIM Sampler:  18%|#8        | 9/50 [00:02<00:12,  3.27it/s]


DDIM Sampler:  20%|##        | 10/50 [00:03<00:11,  3.38it/s]


DDIM Sampler:  22%|##2       |

In [9]:
!mv outputs/txt2img-samples/samples/*.* regularization_images/{dataset}

In [ ]:
# Zip up the files for downloading and reuse.
# Download this file locally so you can reuse during another training on this dataset
!apt-get install -y zip
!zip -r regularization_images.zip regularization_images/{dataset}

In [7]:
# OR download the images from remote
dataset="person_ddim" #@param ["man_euler", "man_unsplash", "person_ddim", "woman_ddim", "blonde_woman"]
!git clone https://github.com/djbielejeski/Stable-Diffusion-Regularization-Images-{dataset}.git

!mv -v Stable-Diffusion-Regularization-Images-{dataset}/{dataset}/*.* ./regularization_images/{dataset}

fatal: destination path 'Stable-Diffusion-Regularization-Images-person_ddim' already exists and is not an empty directory.


renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_00000.png' -> './regularization_images/person_ddim/person_ddim_00000.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_00001.png' -> './regularization_images/person_ddim/person_ddim_00001.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_00002.png' -> './regularization_images/person_ddim/person_ddim_00002.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_00003.png' -> './regularization_images/person_ddim/person_ddim_00003.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_00004.png' -> './regularization_images/person_ddim/person_ddim_00004.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_00005.png' -> './regularization_images/person_ddim/person_ddim_00005.png'
renamed 'Stable-Diffusion-Regularization-Images-pers

renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_00999.png' -> './regularization_images/person_ddim/person_ddim_00999.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_01000.png' -> './regularization_images/person_ddim/person_ddim_01000.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_01001.png' -> './regularization_images/person_ddim/person_ddim_01001.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_01002.png' -> './regularization_images/person_ddim/person_ddim_01002.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_01003.png' -> './regularization_images/person_ddim/person_ddim_01003.png'
renamed 'Stable-Diffusion-Regularization-Images-person_ddim/person_ddim/person_ddim_01004.png' -> './regularization_images/person_ddim/person_ddim_01004.png'
renamed 'Stable-Diffusion-Regularization-Images-pers

# Upload your training images
Upload 10-20 images of someone to

```
/workspace/Dreambooth-Stable-Diffusion/training_images
```

WARNING: Be sure to upload an *even* amount of images, otherwise the training inexplicably stops at 1500 steps.

*   2-3 full body
*   3-5 upper body
*   5-12 close-up on face

The images should be:

- as close as possible to the kind of images you're trying to make

In [7]:
#@title Download and check the images you have just added
import os
import requests
from io import BytesIO
from PIL import Image


def image_grid(imgs, rows, cols):
 assert len(imgs) == rows*cols

 w, h = imgs[0].size
 grid = Image.new('RGB', size=(cols*w, rows*h))
 grid_w, grid_h = grid.size

 for i, img in enumerate(imgs):
  grid.paste(img, box=(i%cols*w, i//cols*h))
 return grid


def getImageFromLocal(filepath):
    with open(filepath, "rb") as fh:
      return Image.open(BytesIO(fh.read())).convert("RGB")

images = list(filter(None,[getImageFromLocal("./inputs/"+filepath) for filepath in os.listdir("./inputs")]))
save_path = "./training_images"
if not os.path.exists(save_path):
 os.mkdir(save_path)
[image.save(f"{save_path}/{i}.png", format="png") for i, image in enumerate(images)]
image_grid(images, 1, len(images))

## Training

If training a person or subject, keep an eye on your project's `logs/{folder}/images/train/samples_scaled_gs-00xxxx` generations.

If training a style, keep an eye on your project's `logs/{folder}/images/train/samples_gs-00xxxx` generations.

In [10]:
# Training

# This isn't used for training, just to help you remember what your trained into the model.
project_name = "kiki"

# MAX STEPS
# How many steps do you want to train for?
max_training_steps = 2000

# Match class_word to the category of the regularization images you chose above.
class_word = "person" # typical uses are "man", "person", "woman"

# This is the unique token you are incorporating into the stable diffusion model.
token = "kiki"


reg_data_root = "./regularization_images/" + dataset

!rm -rf training_images/.ipynb_checkpoints
!python "main.py" \
 --base configs/stable-diffusion/v1-finetune_unfrozen.yaml \
 -t \
 --actual_resume "model.ckpt" \
 --reg_data_root "{reg_data_root}" \
 -n "{project_name}" \
 --gpus 0, \
 --data_root "./training_images" \
 --max_training_steps {max_training_steps} \
 --class_word "{class_word}" \
 --token "{token}" \
 --no-test

^C


## Copy and name the checkpoint file

In [ ]:
# Copy the checkpoint into our `trained_models` folder

directory_paths = !ls -d logs/*
last_checkpoint_file = directory_paths[-1] + "/checkpoints/last.ckpt"
training_images = !find training_images/*
date_string = !date +"%Y-%m-%dT%H-%M-%S"
file_name = date_string[-1] + "_" + project_name + "_" + str(len(training_images)) + "_training_images_" +  str(max_training_steps) + "_max_training_steps_" + token + "_token_" + class_word + "_class_word.ckpt"

file_name = file_name.replace(" ", "_")

!mkdir -p trained_models
!mv "{last_checkpoint_file}" "trained_models/{file_name}"

print("Download your trained model file from trained_models/" + file_name + " and use in your favorite Stable Diffusion repo!")

# Optional - Upload to google drive
* run the following commands in a new `terminal` in the `Dreambooth-Stable-Diffusion` directory
* `chmod +x ./gdrive`
* `./gdrive about`
* `paste your token here after navigating to the link`
* `./gdrive upload trained_models/{file_name.ckpt}`

# Big Important Note!

The way to use your token is `<token> <class>` ie `joepenna person` and not just `joepenna`

## Generate Images With Your Trained Model!

In [ ]:
!python scripts/stable_txt2img.py \
 --ddim_eta 0.0 \
 --n_samples 1 \
 --n_iter 4 \
 --scale 7.0 \
 --ddim_steps 50 \
 --ckpt "/workspace/Dreambooth-Stable-Diffusion/trained_models/{file_name}" \
 --prompt "joepenna person as a masterpiece portrait painting by John Singer Sargent in the style of Rembrandt"